### Importing required libraries

In [1]:
import torch
from torch.utils.data import Dataset
import torchvision #provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.nn as nn

In [2]:
mnist_train_set = torchvision.datasets.MNIST(
            root='./try1',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ])
        )

100.0%


Extracting ./try1\MNIST\raw\train-images-idx3-ubyte.gz to ./try1\MNIST\raw



100.0%


Extracting ./try1\MNIST\raw\train-labels-idx1-ubyte.gz to ./try1\MNIST\raw



100.0%


Extracting ./try1\MNIST\raw\t10k-images-idx3-ubyte.gz to ./try1\MNIST\raw



100.0%

Extracting ./try1\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./try1\MNIST\raw



In [10]:
# trying the sum of 60000 random integers with labels of train dataset of mnist
mnist_train_set.targets + torch.randint(0, 9, (1, 60000)).squeeze()

tensor([ 8,  5,  4,  ..., 10,  9, 14])

In [ ]:
class MNISTWithNumbers(Dataset):
    def __init__(self):
        self.mnist_data = torchvision.datasets.MNIST(
            root='./data',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ])
        )
        self.random_numbers = torch.randint(0, 9, (1, 60000)).squeeze()

    def __getitem__(self, index):
        self.mnist_data
        return 

    def __len__(self):
        return
    
    def train_labels(self):
        return zip(self.mnist_data.targets, self.mnist_data.targets + self.random_numbers) 
